In [5]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


In [2]:
data = {'train': np.load('data/extracted_features/supervised/train.npy'),
       'val': np.load('data/extracted_features/supervised/val.npy'),
       'test': np.load('data/extracted_features/supervised/test.npy')}

In [23]:
sc = StandardScaler()
svm = LinearSVC(max_iter=2000,dual=False,class_weight="balanced")
pipe = Pipeline(steps = [('standardize',sc),('SVM',svm)])

In [28]:
Cs = list(2**np.arange(-19,-3,1,dtype=float))+list(10**np.arange(-7,2,1,dtype=float))

param_grid = {
    "SVM__C": Cs
    }

search = GridSearchCV(pipe, param_grid, n_jobs=-1,cv = 3,verbose=1)
search.fit(data['train'], y_digits)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

NameError: name 'X_digits' is not defined

In [29]:
data['train']

array([[0.18244487, 0.05736828, 0.25183862, ..., 0.07567544, 0.36181074,
        0.0071228 ],
       [0.18310386, 0.35869914, 0.00911823, ..., 0.3779951 , 0.06746776,
        0.00776784],
       [0.0120356 , 0.01036346, 0.12399987, ..., 0.26641172, 0.3992544 ,
        0.99399775],
       ...,
       [0.15099323, 0.6407769 , 0.        , ..., 0.1963284 , 0.0216676 ,
        0.35130203],
       [0.08640178, 0.781633  , 0.47105592, ..., 0.5692706 , 0.0735267 ,
        0.14189433],
       [0.82272375, 0.05622902, 0.00407798, ..., 0.06472108, 0.9037926 ,
        0.        ]], dtype=float32)

In [22]:
10**np.arange(-7,2,1,dtype=float)

array([1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00,
       1.e+01])